In [ ]:
%matplotlib inline
import matplotlib
import numpy as np
import matplotlib.pyplot as plt

import os

import json
import pandas as pd

from dotenv import load_dotenv

load_dotenv(verbose=True, override=True)

import math
import logging
logger = logging.getLogger(__name__)

import dateutil

import requests

from requests.auth import HTTPBasicAuth


In [ ]:
API_KEY = os.getenv('API_KEY')

# electricty
el_serial = os.getenv('el_serial')
MPAN = os.getenv('MPAN')

# gas
gas_serial = os.getenv('gas_serial')
MPRN = os.getenv('MPRN')

PRODUCT = os.getenv('PRODUCT')
TARIFF = os.getenv('TARIFF')


In [ ]:
r = requests.get(f"https://api.octopus.energy/v1/electricity-meter-points/{MPAN}/meters/{el_serial}/consumption/", auth=(API_KEY, ''))
print(r.status_code)

# Download Consumption

In [ ]:
url = f"https://api.octopus.energy/v1/electricity-meter-points/{MPAN}/meters/{el_serial}/consumption/"
cont = True
df = None
while cont:

    print(f'retrieving {url}')
    r = requests.get(url, auth=(API_KEY, ''))
    if df is None:
        print('creating new DF')
        df = pd.DataFrame.from_dict(r.json()['results'])
        # print(f'new size {len(df)}')
    else:
        print('adding to DF')
        # print(f'prior size {len(df)}')
        df = df.append(pd.DataFrame.from_dict(r.json()['results']))
        # print(f'post size {len(df)}')
    if 'next' in r.json() and r.json()['next']:
        url = r.json()['next']
    else:
        cont = False

In [ ]:
df.to_excel('octopus_elect_consumption.xlsx')

# Download Prices

In [ ]:
url = f"https://api.octopus.energy/v1/products/{PRODUCT}/electricity-tariffs/{TARIFF}/standard-unit-rates/"
cont = True
df = None
while cont:

    print(f'retrieving {url}')
    r = requests.get(url)
    if df is None:
        print('creating new DF')
        df = pd.DataFrame.from_dict(r.json()['results'])
        # print(f'new size {len(df)}')
    else:
        print('adding to DF')
        # print(f'prior size {len(df)}')
        df = df.append(pd.DataFrame.from_dict(r.json()['results']))
        # print(f'post size {len(df)}')
    if 'next' in r.json() and r.json()['next']:
        url = r.json()['next']
    else:
        cont = False

In [ ]:
df.to_excel('octopus_agile_rates.xlsx')